In [38]:
import pandas as pd
import numpy as np

In [39]:
submission = pd.read_csv('../data/jpx-tokyo-stock-exchange-prediction/example_test_files/sample_submission.csv')

In [40]:
submission.head(2)

,Date,SecuritiesCode,Rank
0,2021-12-06,1301,0
1,2021-12-06,1332,1


**We observe that even though the spread return of experiment 1 is lower, since its std is also lower, then its sharp ratio is better than exp. 2**

In [41]:

def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """

    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio


### supplemental_files

In [42]:
df = pd.read_csv('../data/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv', parse_dates=["Date"])
df.head(2)

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20211206_1301,2021-12-06,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False,-0.003263
1,20211206_1332,2021-12-06,1332,592.0,599.0,588.0,589.0,1360800,1.0,NaN,False,-0.008993


In [43]:
df['Date'].nunique(), df.Date.max(), df.Date.min()

(56, Timestamp('2022-02-28 00:00:00'), Timestamp('2021-12-06 00:00:00'))

In [29]:
f = df.SecuritiesCode == 1301

pd.DataFrame({'close_diff': list(df[f].Close.diff()) + [np.nan, np.nan], 
              'target': [np.nan, np.nan] + list(df[f].Target.values),
              'Close': [np.nan, np.nan] + list(df[f].Close.values),
              'Open': [np.nan, np.nan] + list(df[f].Open.values),
              'High': [np.nan, np.nan] + list(df[f].High.values)})

,close_diff,target,Close,Open,High
0,NaN,NaN,NaN,NaN,NaN
1,94.0,NaN,NaN,NaN,NaN
2,-10.0,-0.003263,2971.0,2982.0,2982.0
3,30.0,0.009820,3065.0,2998.0,3065.0
4,20.0,0.006483,3055.0,3080.0,3080.0
5,-20.0,-0.006441,3085.0,3050.0,3085.0
6,-25.0,-0.008104,3105.0,3100.0,3105.0
7,30.0,0.009804,3085.0,3110.0,3110.0
8,35.0,0.011327,3060.0,3085.0,3085.0
9,-30.0,-0.009600,3090.0,3060.0,3095.0


### rank stocks by target, disregarding volatility, and consider the higher the return the better

In [45]:
df['Rank'] = df.groupby("Date")["Target"].rank(ascending=False, method="first") - 1

In [46]:
df['Rank'] = df['Rank'].astype('int')
df.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,Rank
0,20211206_1301,2021-12-06,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False,-0.003263,1394
1,20211206_1332,2021-12-06,1332,592.0,599.0,588.0,589.0,1360800,1.0,NaN,False,-0.008993,1670
2,20211206_1333,2021-12-06,1333,2368.0,2388.0,2360.0,2377.0,125900,1.0,NaN,False,-0.009963,1712
3,20211206_1375,2021-12-06,1375,1230.0,1239.0,1224.0,1224.0,81100,1.0,NaN,False,-0.015032,1844
4,20211206_1376,2021-12-06,1376,1339.0,1372.0,1339.0,1351.0,6200,1.0,NaN,False,0.002867,978


**gives the column rank, which indicates the rank of each stock's target for each day among the 2000 stocks for that day.**

**But the submission is asking for the table to be sorted by date, and then within each date it is sorted by rank. From smallest to largest, from 0-1999**

In [47]:
df_submission = df.sort_values(["Date", "Rank"],ascending=True)
df_submission.head(2)

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,Rank
739,20211206_4699,2021-12-06,4699,2660.0,2660.0,2660.0,2660.0,2900,1.0,NaN,True,0.300633,0
1278,20211206_7036,2021-12-06,7036,3800.0,3860.0,3730.0,3765.0,1500,1.0,NaN,False,0.178344,1


In [68]:
portfolio_size = 200
weights = np.linspace(start=2, stop=1, num=portfolio_size)

res = df_submission.groupby('Date').Target.apply(lambda x: ((x[:portfolio_size] * weights).sum() / weights.mean()) - 
                                         ((x[2000 - portfolio_size:] * weights[::-1]).sum() / weights.mean()))
res.mean() / (res.std())

5.4347049198950685

In [70]:
res

(56,)

In [74]:
sampl = np.random.uniform(low=5, high=6, size=(56,))
sampl.std()

0.31225352651075783

In [75]:
sampl.mean() / sampl.std()

17.590473026639035

In [60]:
res.std()

2.789976615745488

In [62]:
np.array([0, 1]).std()

0.5

In [63]:
res.mean() / 0.001

15162.699639984197

and then look at the submission file, you need to sunmission file rank column and our prediction on the correspondence, that is, through the SecuritiesCode column, and then find each day corresponding to the date, df_submisson in the SecuritiesCode corresponding to the rank is how much.

In [58]:
submission

,Date,SecuritiesCode,Rank
0,2021-12-06,1301,0
1,2021-12-06,1332,1
2,2021-12-06,1333,2
3,2021-12-06,1375,3
4,2021-12-06,1376,4
...,...,...,...
111995,2022-02-28,9990,1995
111996,2022-02-28,9991,1996
111997,2022-02-28,9993,1997
111998,2022-02-28,9994,1998


### Calculate the score yourself first, theoretically we don't need to calculate it, the background will calculate it after submitting the file

In [96]:
calc_spread_return_sharpe(df_submission)

5.434704919895069

### Submit to the system

In [69]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for prices, _, _, _, _, sample_prediction in iter_test:
    day_df = df_submission[df_submission['Date'] == prices["Date"].iloc[0]]
    map_dict = day_df.set_index("SecuritiesCode")["Rank"]
    sample_prediction["Rank"] = sample_prediction.SecuritiesCode.map(map_dict)
    env.predict(sample_prediction)

ModuleNotFoundError: No module named 'jpx_tokyo_market_prediction'